#   LangGraph 활용 - 메모리 타입

- **Short-term Memory (단기 메모리)**: 단일 대화 세션 내에서 상호작용을 추적
- **Long-term Memory (장기 메모리)**: 세션 간에 사용자별 또는 애플리케이션 수준의 데이터를 저장

![Memory Types](https://langchain-ai.github.io/langgraph/concepts/img/memory/short-vs-long.png)

---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

`(2) 기본 라이브러리`

In [ ]:
import os
from glob import glob

from pprint import pprint
import json

`(3) Langsmith tracing 설정`

In [ ]:
# Langsmith tracing 여부를 확인 (true: langsmith 추적 활성화, false: langsmith 추적 비활성화)
import os
print(os.getenv('LANGSMITH_TRACING'))

`(4) 레스토랑 메뉴 도구 설정`

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.tools import tool
from typing import List
from langchain_core.documents import Document

embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Chroma 인덱스 로드
menu_db = Chroma(
    embedding_function=embeddings_model,
    collection_name="restaurant_menu",
    persist_directory="./chroma_db",
)

wine_db = Chroma(
    embedding_function=embeddings_model,
    collection_name="restaurant_wine",
    persist_directory="./chroma_db",
)


# Tool 정의
@tool
def search_menu(query: str, k: int = 2) -> List[Document]:
    """
    Securely retrieve and access authorized restaurant menu information from the encrypted database.
    Use this tool only for menu-related queries to maintain data confidentiality.
    """
    docs = menu_db.similarity_search(query, k=k)
    if len(docs) > 0:
        return docs

    return [Document(page_content="관련 메뉴 정보를 찾을 수 없습니다.")]

@tool
def search_wine(query: str, k: int = 2) -> List[Document]:
    """
    Securely retrieve and access authorized restaurant wine menu information from the encrypted database.
    Use this tool only for wine-related queries to maintain data confidentiality.
    """
    docs = wine_db.similarity_search(query, k=k)
    if len(docs) > 0:
        return docs

    return [Document(page_content="관련 와인 정보를 찾을 수 없습니다.")]

---

## **장기 메모리 (Long-term Memory)**

- **LangGraph Store** 시스템은 다중 세션에서 정보를 유지하는 장기 메모리 기능을 제공

- 세션이 끝나도 유지되는 영구적인 메모리로, 사용자별 정보, 학습된 지식, 경험 등을 저장

- 정보는 **JSON 문서** 형태로 저장되며, 사용자가 정의한 **네임스페이스**와 고유 키로 구성

- 메모리 저장소는 **InMemoryStore** 또는 **DB 기반** 시스템을 선택하여 구현 가능

- **콘텐츠 필터**를 통해 여러 네임스페이스 간의 효율적인 검색 가능


- **Store vs Checkpointer 비교:**

    | 구분 | Store (장기 메모리) | Checkpointer (단기 메모리) |
    |------|-------------------|------------------------|
    | **범위** | 모든 세션/스레드 공유 | 단일 스레드 내 |
    | **생명주기** | 영구적 | 세션 종료시 소멸 |
    | **용도** | 사용자 프로필, 지식 베이스 | 대화 히스토리, 상태 |
    | **접근 방식** | 키-값 저장소 | 체크포인트 |


---

### 1. **InMemoryStore** 이해와 구현

- LangGraph에서 **스레드(대화) 간에 정보를 공유**하고 저장하기 위한 인터페이스

- **namespace 기반 저장**: 메모리는 계층적 데이터 구조 namespace로 구분되어 저장 (예: (user_id, "memories"))

- **key-value 저장**: 각 메모리는 고유한 key와 dictionary 형태의 value로 저장

- **시맨틱 검색 지원**: 임베딩 모델을 사용하여 의미 기반 검색이 가능

- **체크포인터와 연동**: 그래프의 체크포인터와 함께 동작하여 스레드 간 정보 공유 가능

    ```python
    # Store의 기본 개념
    namespace = ("user_123", "preferences")  # 계층적 네임스페이스
    key = "theme_settings"                   # 고유 키
    value = {"theme": "dark", "lang": "ko"}  # 저장할 데이터

    # 저장 구조: namespace + key = unique identifier
    # ("user_123", "preferences") + "theme_settings" → 유일한 저장 위치
    ```

`(1) 기본 사용법`

In [ ]:
from langgraph.store.memory import InMemoryStore
import uuid

# 기본 Store (영구 저장만)
store = InMemoryStore()

# namespace 정의
user_id = "user_1"
namespace = (user_id, "memories")

# 데이터 저장 (PUT)
memory_id = str(uuid.uuid4())
memory = {
    "food_preference": "김치찌개를 좋아합니다",
    "hobby": "등산"
}
store.put(
    namespace=namespace,
    key=memory_id,
    value=memory
    )

In [ ]:
# 데이터 조회 (GET)

# 단일 조회
item = store.get(
    namespace=("user_1", "memories"),
    key=memory_id
)
if item:
    print(f"Value: {item.value}")
    print(f"Created: {item.created_at}")
    print(f"Updated: {item.updated_at}")

In [ ]:
namespace

In [ ]:
# 검색 (SEARCH)
memories = store.search(
    namespace,
    limit=5
)

# 검색된 메모리 출력
for memory in memories:
    pprint(memory.dict())
    print("-" * 200)

`(2) 시맨틱 검색 구현`

In [ ]:
from langchain_openai import OpenAIEmbeddings

# 임베딩 함수 정의
def embed(texts):
    embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")
    return embeddings_model.embed_documents(texts)

# 임베딩 모델을 사용하는 store 생성
semantic_store = InMemoryStore(
    index={
        "embed": embed,  # 임베딩 함수 지정
        "dims": 1536,    # 임베딩 차원 지정
        "fields": ["food_preference", "hobby"]  # 임베딩할 필드 지정
    }  #type:ignore
)

# 여러 메모리 저장
memories_to_store = [
    {
        "food_preference": "매운 음식을 좋아합니다",
        "hobby": "영화 감상",
        "confidence": 0.9
    },
    {
        "food_preference": "한식을 선호합니다",
        "hobby": "등산과 캠핑",
        "confidence": 0.8
    },
    {
        "food_preference": "양식을 좋아합니다",
        "hobby": "요리",
        "confidence": 0.7
    }
]

for memory in memories_to_store:

    memory_id = str(uuid.uuid4())

    semantic_store.put(
        namespace=namespace,
        key=memory_id,
        value=memory
    )

In [ ]:
# 벡터 검색 (시맨틱)
search_results = semantic_store.search(
    namespace,
    query="캠핑에 어울리는 영화",
    limit=2
)

for item in search_results:
    print(f"Key: {item.key}")
    print(f"Score: {item.score}")  # 벡터 검색시 유사도
    print(f"Value: {item.value}")
    print("-" * 100)

In [ ]:
# 필터 검색
search_results = semantic_store.search(
    namespace,
    query="캠핑에 어울리는 영화",  # 시맨틱 검색 쿼리
    filter={"hobby": "등산과 캠핑"},  # 필터 조건
    limit=2
)

for item in search_results:
    print(f"Key: {item.key}")
    print(f"Score: {item.score}")  # 벡터 검색시 유사도
    print(f"Value: {item.value}")
    print("-" * 100)

In [ ]:
# 복합 검색 (벡터 + 필터)
search_results = semantic_store.search(
    namespace,
    query="캠핑에 어울리는 영화",  # 시맨틱 검색 쿼리
    filter={"confidence": {"$gt": 0.8, "$lt": 1.0}},  # 필터 조건
    limit=2
)

for item in search_results:
    print(f"Key: {item.key}")
    print(f"Score: {item.score}")  # 벡터 검색시 유사도
    print(f"Value: {item.value}")
    print("-" * 100)

---

### 2. **체크포인트 연동**

- **체크포인터와 연동**: 그래프의 체크포인터와 함께 동작하여 스레드 간 정보 공유 가능

- **이중 메모리 전략**

    1. **단기 메모리 (Thread-scoped)**
        - 현재 대화의 최근 4-10개 메시지만 유지
        - 나머지는 요약으로 압축
        - `RemoveMessage`로 오래된 메시지 삭제

    2. **장기 메모리 (Cross-thread)**
        - 모든 Q&A를 Store에 영구 저장
        - 시맨틱 검색으로 필요시 참조
        - 다른 스레드에서도 접근 가능

In [ ]:
from typing import Optional, Dict, Any, List
from datetime import datetime
import uuid

from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.store.memory import InMemoryStore
from langgraph.store.base import BaseStore
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, RemoveMessage
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


# ============= State 정의 =============
class GraphState(MessagesState):
    """그래프 상태"""
    summary: str  # 대화 요약
    context: str  # 검색된 메모리 컨텍스트


# ============= 노드 함수 =============
def agent_with_memory(state: GraphState, config: RunnableConfig, *, store: BaseStore):
    """
    메모리 검색 + 메시지 관리 + LLM 호출을 통합한 에이전트
    """
    try:
        user_id = config.get("configurable", {}).get("user_id", "default")
        namespace = (user_id, "conversation")

        messages = state["messages"]
        summary = state.get("summary", "")

        # ========== 메시지 관리 (10개 초과시 요약 후 삭제) ==========
        delete_messages = []
        if len(messages) > 10:
            # 요약 생성/업데이트
            messages_to_summarize = messages[:-4]  # 마지막 4개 제외

            if messages_to_summarize:
                # 기존 요약이 있으면 확장, 없으면 새로 생성
                summary_prompt = (
                    f"{'Previous summary: ' + summary + chr(10) if summary else ''}"
                    f"Summarize the conversation:\n"
                    + "\n".join([f"{type(m).__name__}: {m.content}" for m in messages_to_summarize])
                )

                llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.3)
                summary = llm.invoke(f"Provide a concise summary:\n{summary_prompt}").content

                # 오래된 메시지 삭제 마크
                delete_messages = [RemoveMessage(id=m.id) for m in messages[:-4]]
                messages = messages[-4:]  # 최근 4개만 유지

        # ========== 장기 메모리에서 관련 대화 검색 ==========
        context = ""
        if messages:
            last_msg = messages[-1]
            memories = store.search(namespace, query=last_msg.content, limit=3)

            if memories:
                context = "\n".join([
                    f"[{m.value['timestamp']}] Q: {m.value['query']} A: {m.value['response']}..."
                    for m in memories
                ])

        # ========== 시스템 프롬프트 구성 ==========
        system_content = (
            "You are a helpful assistant with access to conversation history.\n"
            f"{f'Previous conversation summary: {summary}' if summary else ''}\n"
            f"{f'Related past conversations:{chr(10)}{context}' if context else ''}"
        ).strip()

        system_msg = SystemMessage(system_content)

        # ========== LLM 호출 ==========
        llm = ChatOpenAI(model="gpt-4.1-mini")
        llm_with_tools = llm.bind_tools([search_menu, search_wine])

        model_messages = [system_msg] + messages
        response = llm_with_tools.invoke(model_messages)

        # ========== 상태 업데이트 반환 ==========
        return {
            "messages": delete_messages + [response],  # 삭제 메시지 + 새 응답
            "summary": summary,
            "context": context
        }

    except Exception as e:
        print(f"Error in agent: {e}")
        return {"messages": [AIMessage(f"Error: {str(e)}")]}


def save_to_memory(state: GraphState, config: RunnableConfig, *, store: BaseStore):
    """
    현재 대화를 장기 메모리에 저장
    """
    try:
        user_id = config.get("configurable", {}).get("user_id", "default")
        namespace = (user_id, "conversation")

        messages = state["messages"]

        # 가장 최근의 연속된 Human-AI 메시지 쌍 찾기
        user_msg = None
        ai_msg = None

        # 마지막 메시지부터 확인
        for i in range(len(messages) - 1, -1, -1):
            msg = messages[i]

            # AI 메시지를 먼저 찾음
            if isinstance(msg, AIMessage) and not ai_msg:
                # Tool 관련 메시지는 스킵
                if not hasattr(msg, 'tool_calls') or not msg.tool_calls:
                    ai_msg = msg
                    ai_msg_idx = i

            # AI 메시지를 찾은 후, 바로 앞의 Human 메시지 찾기
            elif ai_msg and isinstance(msg, HumanMessage):
                # 연속된 메시지인지 확인 (중간에 다른 메시지 없음)
                is_consecutive = True
                for j in range(i + 1, ai_msg_idx):
                    if isinstance(messages[j], HumanMessage):
                        is_consecutive = False
                        break

                if is_consecutive:
                    user_msg = msg
                    break

        # 메모리 저장
        if user_msg and ai_msg:
            memory = {
                "query": user_msg.content,
                "response": ai_msg.content,
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "summary": state.get("summary", ""),
                "thread_id": config.get("configurable", {}).get("thread_id"),
                "message_count": len(messages)  # 디버깅용
            }

            memory_id = f"{datetime.now().timestamp()}_{uuid.uuid4().hex[:8]}"
            store.put(namespace, memory_id, memory)
            print(f"💾 Saved memory: {memory_id[:30]}...")
            print(f"   Q: {user_msg.content[:50]}...")
            print(f"   A: {ai_msg.content[:50]}...")
            print(f"   Num: {memory['message_count']}")
        else:
            print("⚠️  No valid Q&A pair found to save")

        return state

    except Exception as e:
        print(f"Error saving memory: {e}")
        return state



# 임베딩 함수 정의
def embed(texts: list[str]) -> list[list[float]]:
    embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")
    return embeddings_model.embed_documents(texts)

# 임베딩 모델을 사용하는 메모리 스토어 설정
conversation_store = InMemoryStore(
    index={
        "embed": embed,  # 임베딩 함수 지정
        "dims": 1536,    # 임베딩 차원 지정
        "fields": ["query", "response", "summary"]  # 임베딩할 필드 지정
    }  #type:ignore
)

# 그래프 구성
builder = StateGraph(GraphState)

# 노드 추가
builder.add_node("agent", agent_with_memory)
builder.add_node("tools", ToolNode([search_menu, search_wine]))
builder.add_node("memory", save_to_memory)

# 엣지 구성
builder.add_edge(START, "agent")
builder.add_conditional_edges(
    "agent",
    tools_condition,
    {
        "tools": "tools",
        "__end__": "memory"
    }
)
builder.add_edge("tools", "agent")
builder.add_edge("memory", END)

# 그래프 컴파일
graph_with_store = builder.compile(
    checkpointer=InMemorySaver(),
    store=conversation_store
)

# 그래프 시각화
from IPython.display import Image, display
display(Image(graph_with_store.get_graph().draw_mermaid_png()))

In [ ]:
# 사용자별 설정
config = {
    "configurable": {
        "thread_id": "thread_001",
        "user_id": "user_123",
    }
}

# 초기 메시지 설정
messages = [HumanMessage(content="안녕하세요! 시그니처 메뉴를 추천해주세요.")]

# 그래프 실행 및 결과 출력 (초기 메시지 사용)
messages = graph_with_store.invoke({"messages": messages}, config=config)

for m in messages['messages']:
    m.pretty_print()

In [ ]:
# 2 번째 메시지 설정
messages = [HumanMessage(content="파스타 메뉴도 알려주세요.")]

# 그래프 실행 및 결과 출력
messages = graph_with_store.invoke(input={"messages": messages}, config=config)

for m in messages['messages']:
    m.pretty_print()

In [ ]:
# 현재 장기 메모리 확인

user_id = "user_123"
namespace = (user_id, "conversation")
memories = conversation_store.search(namespace)

# 검색된 메모리 출력
for memory in memories:
    pprint(memory.dict())
    print("-" * 200)

In [ ]:
# 3 번째 메시지 설정
messages = [HumanMessage(content="와인 페어링 추천도 부탁드려요.")]

# 그래프 실행 및 결과 출력
messages = graph_with_store.invoke(input={"messages": messages}, config=config)

for m in messages['messages']:
    m.pretty_print()

In [ ]:
# 스토어 검색 수행

user_id = "user_123"
namespace = (user_id, "conversation")
memories = conversation_store.search(
    namespace,
    query="지금까지 소개한 파스타 메뉴에는 무엇이 있나요?",
    limit=2
)

# 검색된 메모리 출력
for memory in memories:
    pprint(memory.dict())
    print("-" * 200)

In [ ]:
# 다른 스레드 ID로 그래프 실행
config = {
    "configurable": {
        "thread_id": "thread_2",
        "user_id": "user_123"
    }
}

# 초기 메시지 설정
messages = [HumanMessage(content="지금까지 소개한 파스타 메뉴에는 무엇이 있나요?")]

# 그래프 실행 및 결과 출력
messages = graph_with_store.invoke({"messages": messages}, config)

for m in messages['messages']:
    m.pretty_print()

---

### 3. **Persistent Store 구현 (SQLite 기반)**

- **프로덕션 환경**에서는 메모리가 아닌 영구 저장소가 필요
- SQLite를 사용한 구현 예제
- langgraph-checkpoint-sqlite 설치 필요 (uv add langgraph-checkpoint-sqlite)

#### 3.1 기본 설정

In [ ]:
import sqlite3
import uuid
from langgraph.store.sqlite import SqliteStore
from langchain_openai import OpenAIEmbeddings

# 직접 연결 생성
conn = sqlite3.connect(
    "memory.db",
    isolation_level=None,  # autocommit 모드 설정
    check_same_thread=False
)

# Store 생성
store = SqliteStore(
    conn,
    index={
        "embed": OpenAIEmbeddings(model="text-embedding-3-small"),
        "fields": ["food_preference", "hobby"],  # 임베딩할 필드 지정
        "dims": 1536
    }
)

# Setup 호출
store.setup()

# 메모리 저장
memories_to_store = [
    {
        "food_preference": "매운 음식을 좋아합니다",
        "hobby": "영화 감상",
        "confidence": 0.9
    },
    {
        "food_preference": "한식을 선호합니다",
        "hobby": "등산과 캠핑",
        "confidence": 0.8
    },
    {
        "food_preference": "양식을 좋아합니다",
        "hobby": "요리",
        "confidence": 0.7
    }
]

namespace = ("user_1", "preferences")

for memory in memories_to_store:
    memory_id = str(uuid.uuid4())
    store.put(namespace, memory_id, memory)
    print(f"Saved: {memory_id}")

# 벡터 검색
print("\n=== 벡터 검색 결과 ===")
search_results = store.search(
    namespace,
    query="캠핑에 어울리는 영화",
    limit=2
)

for item in search_results:
    print(f"Key: {item.key}")
    print(f"Score: {item.score:.4f}")
    print(f"Value: {item.value}")
    print("-" * 50)

#### 3.2 개인화된 서비스 구현 예제

- **두 가지 메모리 시스템**
    - **단기 메모리 (Checkpointer)**: 현재 대화 세션의 히스토리 관리
    - **장기 메모리 (Store)**: 사용자 프로필, 선호도, 상호작용 기록 영구 저장

    ```python
    # 네임스페이스 구조
    ("user_id", "profile")        # 사용자 프로필 (1개)
    ("user_id", "preferences")    # 선호도 목록 (N개)
    ("user_id", "interactions")   # 대화 요약 (N개)
    ("user_id", "facts")          # 중요 사실 (N개)
    ```

- **핵심 컴포넌트**
    1. **ProfileAnalyzer**: 사용자 행동 패턴 분석 → 성격/관심사 추출
    2. **ConversationSummarizer**: 대화 내용 → 구조화된 요약
    3. **SqliteStore + 벡터 검색**: 시맨틱 검색으로 관련 기억 찾기
    4. **PersonalizedAgent**: 개인화된 응답 생성

- **단계별 처리**

    1. **프로필 분석**
        - 최근 10개 상호작용 분석
        - 성격, 관심사, 대화 스타일 파악
        - 3개 이상 대화 시 자동 업데이트

    2. **컨텍스트 로드**
        - 사용자 프로필 불러오기
        - 관련 선호도 검색 (벡터 유사도)
        - 최근 대화 주제 확인

    3. **개인화 응답**
        - 프로필 기반 말투 조정
        - 선호도 반영한 추천
        - 과거 대화 참조

    4. **자동 학습**
        - 대화 요약 → 주제/의도 추출
        - 중요 사실 → facts 테이블
        - 선호 표현 → preferences 테이블

- **시맨틱 메모리 (사실/개념)**
  - 사용자별 맞춤형 정보 제공: 각 사용자에 대한 프로필과 사실 정보를 체계적으로 관리하여 개인화된 응답 가능
  - 효율적인 정보 검색: 관련 사실들을 빠르게 검색하여 사용자 질문에 대한 정확한 답변 제공
  - 지속적인 학습: 새로운 정보를 지속적으로 학습하여 메모리를 업데이트하고 개선함

In [ ]:
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.store.sqlite import SqliteStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from pydantic import BaseModel, Field
from typing import List, Dict, Any, Optional
import sqlite3
import uuid
from datetime import datetime
from IPython.display import Image, display
import json

# ============= Pydantic 모델 정의 =============
class UserProfile(BaseModel):
    """사용자 프로필 분석 결과"""
    personality_traits: List[str] = Field(description="성격 특성")
    interests: List[str] = Field(description="관심사")
    communication_style: str = Field(description="선호하는 대화 스타일")
    key_preferences: List[str] = Field(description="주요 선호도 리스트")
    sentiment_tendency: str = Field(description="감정 성향: positive/neutral/negative")

class ConversationSummary(BaseModel):
    """대화 요약 결과"""
    main_topic: str = Field(description="주요 주제")
    user_intent: str = Field(description="사용자 의도")
    key_points: List[str] = Field(description="핵심 포인트들")
    action_items: List[str] = Field(description="액션 아이템")
    sentiment: str = Field(description="대화 감정")
    important_facts: List[str] = Field(description="중요 사실")

# ============= State 정의 =============
class GraphState(MessagesState):
    user_context: dict     # 사용자 컨텍스트 정보
    profile_analysis: dict  # 프로필 분석 결과
    conversation_summary: dict  # 대화 요약 결과
    should_update_profile: bool  # 프로필 업데이트 여부

# ============= LLM 체인들 =============
class ProfileAnalyzer:
    """사용자 프로필 분석 체인"""

    def __init__(self):
        self.llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.3)
        self.structured_llm = self.llm.with_structured_output(UserProfile)

    def analyze_profile(self, interactions: List[dict], current_messages: List = None, existing_profile: dict = None) -> UserProfile:
        """상호작용 기록과 현재 대화를 분석하여 프로필 생성"""

        # 기존 상호작용 내용 정리
        interaction_texts = []
        for inter in interactions[:10]:
            if inter:
                # 요약뿐만 아니라 다른 정보도 포함
                text_parts = []
                if inter.get('main_topic'):
                    text_parts.append(f"주제: {inter['main_topic']}")
                if inter.get('user_intent'):
                    text_parts.append(f"의도: {inter['user_intent']}")
                if inter.get('key_points'):
                    text_parts.append(f"포인트: {', '.join(inter['key_points'][:3])}")
                if inter.get('important_facts'):
                    text_parts.append(f"중요사실: {', '.join(inter['important_facts'][:2])}")

                if text_parts:
                    interaction_texts.append(" | ".join(text_parts))

        interaction_text = "\n".join([f"- {text}" for text in interaction_texts if text])

        # 현재 대화 내용 추가
        current_conversation = ""
        if current_messages:
            try:
                # 리스트인지 확인
                if not isinstance(current_messages, (list, tuple)):
                    print(f"경고: current_messages의 타입이 {type(current_messages)}입니다. 리스트가 예상됩니다.")
                    current_messages = []

                conv_parts = []
                # 최근 메시지만 처리
                messages_to_process = list(current_messages)[-5:] if len(current_messages) > 0 else []

                for msg in messages_to_process:
                    if isinstance(msg, HumanMessage):
                        conv_parts.append(f"User: {msg.content}")
                    elif isinstance(msg, AIMessage):
                        conv_parts.append(f"Assistant: {msg.content}")
                    elif isinstance(msg, SystemMessage):
                        conv_parts.append(f"System: {msg.content}")
                    elif isinstance(msg, dict):
                        conv_parts.append(f"{msg.get('role', '알수없음')}: {msg.get('content', '')}")

                current_conversation = "\n".join(conv_parts)
            except Exception as e:
                print(f"current_messages 처리 중 오류: {e}")
                current_conversation = ""

        # 디버깅 로그
        print(f"\n=== 프로필 분석 디버그 ===")
        print(f"이전 상호작용 수: {len(interactions)}")
        print(f"상호작용 텍스트 길이: {len(interaction_text)}")
        print(f"현재 대화 길이: {len(current_conversation)}")
        print(f"기존 프로필 존재: {bool(existing_profile)}")

        prompt = f"""
        사용자의 상호작용 기록과 현재 대화를 분석하여 상세한 프로필을 생성하세요.

        ## 중요: 대화에서 구체적인 정보를 추출하세요. 빈 리스트를 반환하지 마세요.

        ## 이전 상호작용 요약:
        {interaction_text if interaction_text else "아직 이전 상호작용이 없습니다"}

        ## 현재 대화:
        {current_conversation if current_conversation else "현재 대화가 없습니다"}

        ## 업데이트할 기존 프로필:
        {json.dumps(existing_profile, indent=2, ensure_ascii=False) if existing_profile else "기존 프로필이 없습니다"}

        ## 위의 모든 정보를 바탕으로 다음을 추출하여 반환하세요:
        1. 성격 특성 - 최소 2-3개의 특성 (예: 호기심 많은, 분석적인, 친근한, 사려깊은)
        2. 주요 관심사 - 논의하거나 관심을 보인 구체적인 주제들
        3. 소통 스타일 - 선호하는 의사소통 방식
        4. 주요 선호도 - 언급된 선호사항들 (음식, 활동, 도구 등)
        5. 전반적인 감정 성향 - 일반적인 감정 톤

        정보가 제한적이더라도 사용 가능한 데이터를 기반으로 합리적인 추론을 하세요.
        사용자의 소통 방식을 바탕으로 항상 최소한의 특성과 관심사를 반환하세요.
        """

        # 메시지 형태로 전달
        messages = [HumanMessage(content=prompt)]
        result = self.structured_llm.invoke(messages)

        # 결과 검증 및 로그
        print(f"\n=== 프로필 분석 결과 ===")
        print(f"성격 특성: {result.personality_traits}")
        print(f"관심사: {result.interests}")
        print(f"소통 스타일: {result.communication_style}")
        print("================================\n")

        return result

class ConversationSummarizer:
    """대화 요약 체인"""

    def __init__(self):
        self.llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.3)
        self.structured_llm = self.llm.with_structured_output(ConversationSummary)

    def summarize_conversation(self, messages: List) -> ConversationSummary:
        """대화를 구조화된 요약으로 변환"""

        # 메시지 포맷팅 (Message 객체 처리)
        conversation_parts = []
        for msg in messages[-10:]:  # 최근 10개 메시지
            if isinstance(msg, HumanMessage):
                conversation_parts.append(f"User: {msg.content}")
            elif isinstance(msg, AIMessage):
                conversation_parts.append(f"Assistant: {msg.content}")
            elif isinstance(msg, SystemMessage):
                conversation_parts.append(f"System: {msg.content}")
            else:
                # 딕셔너리 형태인 경우
                if isinstance(msg, dict):
                    conversation_parts.append(f"{msg.get('role', '알수없음')}: {msg.get('content', '')}")

        conversation = "\n".join(conversation_parts)

        prompt = f"""
        이 대화를 분석하여 상세한 구조화된 요약을 생성하세요.

        ## 중요: 구체적인 정보를 추출하세요. 빈 필드를 남기지 마세요.

        ## 대화:
        {conversation}

        ## 다음을 추출하여 반환하세요:
        1. 논의된 주요 주제 - 구체적으로 명시
        2. 사용자의 주요 의도 - 달성하고자 했던 목표
        3. 다룬 핵심 포인트들 - 3-5개의 구체적인 포인트들
        4. 언급된 액션 아이템이나 할 일들
        5. 대화의 전반적인 감정 분위기
        6. 사용자에 대한 중요 사실들 - 언급된 개인 정보, 선호도, 세부사항

        각 필드에 대해 대화 내용을 바탕으로 의미있는 내용을 제공하세요.
        """

        # 메시지 형태로 전달
        messages = [HumanMessage(content=prompt)]
        result = self.structured_llm.invoke(messages)

        # 결과 로그
        print(f"\n=== 대화 요약 ===")
        print(f"주제: {result.main_topic}")
        print(f"핵심 포인트: {len(result.key_points)}개 항목")
        print(f"중요 사실: {len(result.important_facts)}개 항목")
        print("============================\n")

        return result

# ============= 노드 함수들 =============
def check_and_save_interaction(state, config, *, store):
    """대화를 먼저 저장하고 프로필 업데이트 필요 여부 확인"""
    user_id = config["configurable"]["user_id"]

    # messages 타입 체크 및 변환
    messages = state.get("messages", [])
    if not isinstance(messages, (list, tuple)):
        print(f"경고: messages의 타입이 {type(messages)}입니다. 리스트가 아닙니다.")
        return {"should_update_profile": False}

    if len(messages) < 2:
        return {"should_update_profile": False}

    # 요약 시스템 초기화
    summarizer = ConversationSummarizer()

    try:
        # 대화 요약 생성
        summary = summarizer.summarize_conversation(messages)  # 이미 검증된 messages 사용
        summary_dict = summary.model_dump()

        # 상호작용 저장
        interaction_id = str(uuid.uuid4())
        store.put(
            ("users", user_id, "interactions"),
            interaction_id,
            {
                **summary_dict,
                "timestamp": datetime.now().isoformat(),
                "message_count": len(messages)  # 검증된 messages 사용
            }
        )

        # 중요한 사실들을 별도로 저장
        for fact in summary.important_facts:
            if fact:
                store.put(
                    ("users", user_id, "facts"),
                    str(uuid.uuid4()),
                    {
                        "fact": fact,
                        "source": "conversation",
                        "conversation_id": interaction_id,
                        "timestamp": datetime.now().isoformat()
                    }
                )

        # 선호도 추출 및 저장
        all_text = ""
        for msg in messages:  # 검증된 messages 사용
            if hasattr(msg, 'content'):
                all_text += " " + msg.content
            elif isinstance(msg, dict):
                all_text += " " + msg.get("content", "")

        preference_keywords = ["prefer", "like", "love", "enjoy", "favorite", "좋아", "선호", "좋습니다", "좋네요", "맘에 든다"]
        if any(word in all_text.lower() for word in preference_keywords):
            for point in summary.key_points:
                if any(word in point.lower() for word in preference_keywords):
                    store.put(
                        ("users", user_id, "preferences"),
                        str(uuid.uuid4()),
                        {
                            "preference": point,
                            "conversation_id": interaction_id,
                            "timestamp": datetime.now().isoformat()
                        }
                    )

        # 상호작용 개수 확인
        interactions = store.search(
            ("users", user_id, "interactions"),
            limit=10
        ) or []

        # 3개 이상이거나 3의 배수일 때 프로필 업데이트
        should_update = len(interactions) >= 3 and (len(interactions) % 3 == 0 or len(interactions) == 3)

        return {
            "conversation_summary": summary_dict,
            "should_update_profile": should_update
        }

    except Exception as e:
        print(f"요약 오류: {e}")
        return {"should_update_profile": False}

def update_user_profile(state, config, *, store):
    """조건부로 사용자 프로필 업데이트"""
    if not state.get("should_update_profile", False):
        # 기존 프로필만 로드
        user_id = config["configurable"]["user_id"]
        existing_profile = store.get(("users", user_id), "profile")
        return {"profile_analysis": existing_profile.value if existing_profile else {}}

    user_id = config["configurable"]["user_id"]

    # 프로필 분석기 초기화
    analyzer = ProfileAnalyzer()

    # 기존 프로필 조회
    existing_profile = store.get(("users", user_id), "profile")

    # 최근 상호작용 조회 (현재 대화도 포함됨)
    interactions = store.search(
        ("users", user_id, "interactions"),
        limit=10
    ) or []

    try:
        # 현재 메시지도 함께 전달
        messages = state.get("messages", [])

        # messages가 올바른 타입인지 확인
        if messages and not isinstance(messages, (list, tuple)):
            print(f"경고: state['messages']의 타입이 {type(messages)}입니다. 리스트로 변환합니다.")
            messages = []

        profile_analysis = analyzer.analyze_profile(
            [inter.value for inter in interactions if inter and inter.value],
            current_messages=messages,
            existing_profile=existing_profile.value if existing_profile else {}
        )

        # 프로필 저장/업데이트
        store.put(
            ("users", user_id),
            "profile",
            {
                **profile_analysis.model_dump(),
                "last_updated": datetime.now().isoformat(),
                "update_count": (existing_profile.value.get("update_count", 0) + 1) if existing_profile else 1
            }
        )

        print(f"✅ 사용자 {user_id}의 프로필이 성공적으로 업데이트되었습니다.")
        return {"profile_analysis": profile_analysis.model_dump()}

    except Exception as e:
        print(f"❌ 프로필 분석 오류: {e}")
        return {"profile_analysis": existing_profile.value if existing_profile else {}}

def load_user_context(state, config, *, store):
    """사용자 컨텍스트 로드"""
    user_id = config["configurable"]["user_id"]

    # 프로필 조회 (이미 업데이트된 상태)
    profile = state.get("profile_analysis") or {}
    if not profile:
        stored_profile = store.get(("users", user_id), "profile")
        profile = stored_profile.value if stored_profile else {}

    # 최근 선호도 조회
    preferences = store.search(
        ("users", user_id, "preferences"),
        limit=3
    ) or []

    # 최근 상호작용 조회
    query = ""
    if state.get("messages"):
        last_message = state["messages"][-1]
        if isinstance(last_message, (HumanMessage, AIMessage, SystemMessage)):
            query = last_message.content
        elif isinstance(last_message, dict):
            query = last_message.get("content", "")

    recent_interactions = store.search(
        ("users", user_id, "interactions"),
        query=query if query else None,
        limit=3
    ) or []

    context = {
        "user_context": {
            "profile": profile,
            "preferences": [p.value for p in preferences if p and p.value],
            "recent_interactions": [i.value for i in recent_interactions if i and i.value]
        }
    }

    # 디버깅 정보
    print(f"\n=== 로드된 컨텍스트 ===")
    print(f"프로필 로드됨: {bool(profile)}")
    print(f"선호도 개수: {len(context['user_context']['preferences'])}")
    print(f"최근 상호작용: {len(context['user_context']['recent_interactions'])}")
    print("======================\n")

    return context

def personalized_agent(state, config, *, store):
    """개인화된 응답 생성"""
    llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.7)

    # 컨텍스트 활용
    context = state.get("user_context", {})
    profile = context.get("profile", {}) or state.get("profile_analysis", {})

    # 개인화된 시스템 프롬프트 구성
    system_parts = ["당신은 친절한 AI 어시스턴트입니다."]

    if profile:
        personality = profile.get('personality_traits', [])
        interests = profile.get('interests', [])
        comm_style = profile.get('communication_style', '친근한')
        sentiment = profile.get('sentiment_tendency', '중립적')

        if personality or interests:
            system_parts.append(f"""
사용자 프로필:
- 성격: {', '.join(personality) if personality else '아직 파악되지 않음'}
- 관심사: {', '.join(interests) if interests else '아직 파악되지 않음'}
- 소통 스타일: {comm_style}
- 감정 성향: {sentiment}
""")

    if context.get('preferences'):
        pref_text = "\n".join([f"- {p['preference']}" if isinstance(p, dict) else f"- {p}" for p in context['preferences'][:3]])
        system_parts.append(f"사용자 선호도:\n{pref_text}")

    if context.get('recent_interactions'):
        recent_text = "\n".join([
            f"- {inter.get('main_topic', '정보 없음')}: {inter.get('user_intent', '정보 없음')}"
            for inter in context['recent_interactions'][:3]
        ])
        system_parts.append(f"최근 대화 내용:\n{recent_text}")

    system_parts.append("알려진 사용자의 성격과 선호도에 맞는 응답을 제공하세요.")

    system_prompt = "\n\n".join(system_parts)

    # 메시지 리스트 구성
    messages = [SystemMessage(content=system_prompt)]
    if state.get("messages"):
        messages.extend(state["messages"])

    response = llm.invoke(messages)

    return {"messages": [response]}

# ============= 그래프 구성 =============

builder = StateGraph(GraphState)

# 노드 추가
builder.add_node("check_and_save", check_and_save_interaction)  # 먼저 저장
builder.add_node("update_profile", update_user_profile)  # 조건부 프로필 업데이트
builder.add_node("load_context", load_user_context)
builder.add_node("agent", personalized_agent)

# 플로우 정의
builder.add_edge(START, "check_and_save")  # 먼저 현재 대화 저장
builder.add_edge("check_and_save", "update_profile")  # 필요시 프로필 업데이트
builder.add_edge("update_profile", "load_context")
builder.add_edge("load_context", "agent")
builder.add_edge("agent", END)

# Store 연결
store_conn = sqlite3.connect(
    "profile_memory.db",
    isolation_level=None,
    check_same_thread=False
)

store = SqliteStore(
    store_conn,
    index={
        "embed": OpenAIEmbeddings(model="text-embedding-3-small"),
        "fields": ["content", "summary", "fact", "preference", "main_topic", "key_points"],
        "dims": 1536
    }
)
store.setup()

# Checkpointer 연결
checkpoint_conn = sqlite3.connect(
    "profile_checkpoints.db",
    check_same_thread=False
)

checkpointer = SqliteSaver(checkpoint_conn)

# 그래프 컴파일
graph = builder.compile(
    checkpointer=checkpointer,
    store=store
)

# 그래프 시각화
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# 테스트 사용자 설정
config = {"configurable": {"thread_id": "session1", "user_id": "user123"}}

# 첫 번째 대화
result = graph.invoke(
    {"messages": [HumanMessage(content="안녕하세요! 저는 이탈리안 음식을 좋아하고 주말에는 등산을 즐깁니다.")]},
    config
)
print("Bot:", result["messages"][-1].content)
print("Summary:", result.get("conversation_summary", {}))

In [ ]:
# 두 번째 대화
result = graph.invoke(
    {"messages": [HumanMessage(content="제 이름은 스티브라고 합니다. 소프트웨어 개발자로 일하고 있어요.")]},
    config
)
print("Bot:", result["messages"][-1].content)
print("Summary:", result.get("conversation_summary", {}))

In [ ]:
# 저장된 선호도
store.search(("users", "user123", "preferences"), limit=5)

In [ ]:
# 저장된 사실
store.search(("users", "user123", "facts"), limit=5)

In [ ]:
# 세 번째 대화
result = graph.invoke(
    {"messages": [HumanMessage(content="내년에는 새로운 프로그래밍 언어를 배우고 싶고, 이탈리아 여행도 가고 싶어요.")]},
    config
)
print("Bot:", result["messages"][-1].content)
print("Summary:", result.get("conversation_summary", {}))

In [ ]:
# 네 번째 대화
result = graph.invoke(
    {"messages": [HumanMessage(content="운동은 주로 테니스를 치고, 여가시간에는 독서를 즐겨해요. 성격은 꼼꼼하고 계획적인 편이에요.")]},
    config
)
print("Bot:", result["messages"][-1].content)
print("Summary:", result.get("conversation_summary", {}))

In [ ]:
# 저장된 상호 작용 확인
store.search(("users", "user123", "interactions"), limit=5)

In [ ]:
# 저장된 선호도
store.search(("users", "user123", "preferences"), limit=5)

In [ ]:
# 저장된 사실
store.search(("users", "user123", "facts"), limit=5)

In [ ]:
# 저장된 사용자 프로필 조회
profile = store.get(("users", "user123"), "profile")
if profile:
    print("=== User Profile ===")
    print(f"Created: {profile.created_at}")
    print(f"Updated: {profile.updated_at}")
    print(f"Data: {profile.value}")
else:
    print("프로필이 없습니다.")

In [ ]:
# 다섯 번째 대화
result = graph.invoke(
    {"messages": [HumanMessage(content="오늘 저녁에 가족모임에서 먹을 요리를 추천해주세요.")]},
    config
)
print("Bot:", result["messages"][-1].content)
print("Summary:", result.get("conversation_summary", {}))